In [1]:
# import packages
import pandas as pd, numpy as np
import json
# import files
import data as dt

1. Replicar 100 bps? -> **10 Basis points: dejar como parametro del backtest**
2. Determinar origin vs destination exchange -> **Kraken es origin, Bitfinex es destination**
3. Como replicar los niveles del origin en el destination (volumen)
4. Validar el método para determinar maker/taker fee: iterativo vs estático
5. Orden de llenado por tiempo de entrada.
6. Evolución de Orderbooks tipo bootstrap.

In [2]:
# load data
ob_krak, ob_bit = dt.read_jsonOB(file_name = 'orderbooks_05jul21.json')


# Alineacion de timestamps

In [5]:
list(ob_krak.keys())[0], list(ob_bit.keys())[0], list(ob_bit.keys())[1], list(ob_bit.keys())[2], list(ob_bit.keys())[3]

('2021-07-05T13:06:49.495Z',
 '2021-07-05T13:06:46.571Z',
 '2021-07-05T13:06:47.918Z',
 '2021-07-05T13:06:49.414Z',
 '2021-07-05T13:06:51.077Z')

In [6]:
# filtro  para que destination sea despues de origin
aux = sum(pd.to_datetime(pd.Series((ob_bit.keys()))) < pd.to_datetime(pd.Series(ob_krak.keys()))[0])
[ob_bit.pop(key) for key in list(ob_bit.keys())[:aux]];

# Origin exchange (Kraken)

In [7]:
# OB 1
df_krak = ob_krak[list(ob_krak.keys())[0]]
# midprice
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
display(df_krak.head(3))
mid_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074


28298.35

In [8]:
upper_krak = mid_krak*1.001
lower_krak = mid_krak*0.999
(lower_krak, upper_krak)

(28270.051649999998, 28326.648349999996)

In [9]:
bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]

In [10]:
[d.shape for d in [df_krak, bids_krak, asks_krak]]

[(100, 4), (30, 2), (23, 2)]

In [11]:
levels = pd.concat([bids_krak, asks_krak], axis=1)

In [12]:
levels

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074
3,28292.2,0.011,28305.5,0.123
4,28292.0,1.221,28307.3,0.204
5,28291.2,0.745,28308.2,0.005
6,28291.0,0.396,28310.0,0.004
7,28290.9,0.340,28311.8,0.745
8,28290.8,0.006,28314.9,0.250
9,28290.0,0.003,28315.2,5.000


# Destination exchange (Bitfinex)

In [13]:
# OB 1
df_bit = ob_bit[list(ob_bit.keys())[0]]
# midprice
mid_bit = (df_bit['bid'][0]+df_bit['ask'][0])/2
display(df_bit.head(3))
mid_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


28276.5

# Añadir niveles a destination

### Bids

In [14]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [15]:
# merge levels to create new OB
bids = df_bit[['bid_size','bid']].merge(levels[['bid_size','bid']], how='outer', on='bid'
                                       ).sort_values(by='bid', ascending=False, ignore_index=True)
bids.rename(columns={'bid_size_x':'bid_size','bid_size_y':'add_size'}, inplace=True)

In [16]:
# identify each order's fee structure
bids['fee_type'] = ['taker' if bid > df_bit['ask'][0] else 'maker' if pd.notna(add_size) else np.nan
                    for bid, add_size in zip(bids['bid'].values, bids['add_size'].values)]

bids.fillna(0, inplace=True)

In [17]:
# compute total volume in each OB level
bids['new_size'] = bids['bid_size'].values+bids['add_size'].values

In [18]:
df_bit[['ask','ask_size']].iloc[0,:]

ask         28278.000000
ask_size        0.011063
Name: 0, dtype: float64

In [19]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [23]:
df_bit.shape[0],levels.shape[0], bids.shape[0]

(25, 30, 55)

# Preliminary approach using loops

In [342]:
bit_ask = df_bit[['ask','ask_size']]
bit_bid = df_bit[['bid_size','bid']]

In [343]:
bid_levels = levels[['bid_size','bid']]
bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
bid_levels.rename(columns={'bid_size':'size',
                          'bid':'price',
                          'bid_flag':'type'}, inplace=True)

ask_levels = levels[['ask_size','ask']]
ask_levels['ask_flag'] = ['ask']*ask_levels.shape[0]
ask_levels.rename(columns={'ask_size':'size',
                          'ask':'price',
                          'ask_flag':'type'}, inplace=True)

levels_added = bid_levels.append(ask_levels, ignore_index=True).dropna()

levels_added['transaction_time'] = np.random.uniform(1250, size=levels_added.shape[0])
levels_added.sort_values(by='transaction_time', inplace=True, ignore_index=True)

<ipython-input-343-56ab3a99147f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
<ipython-input-343-56ab3a99147f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ask_levels['ask_flag'] = ['ask']*ask_levels.shape[0]


In [344]:
for i in range(levels_added.shape[0]):

    to_fill = levels_added.iloc[0,:]
    
    if to_fill['type']=='bid':
        # identify order's fee structure
        fee_type = 'taker' if to_fill['price'] > bit_ask.iloc[0]['ask'] else 'maker'


        if fee_type=='taker':

            bit_ask['accum_size'] = bit_ask['ask_size'].cumsum()


            vol_to_fill=to_fill['size']

            bit_ask['remaining_vol'] = vol_to_fill - bit_ask['accum_size']
            to_drop = bit_ask[bit_ask['remaining_vol']>0]

            bit_ask.drop(to_drop.index, inplace=True)
            bit_ask.reset_index(drop=True, inplace=True)

            new_vol = -bit_ask.iloc[0,-1]  # remaing volume on surviving level

            bit_ask.iloc[0,1] = new_vol
            bit_ask.drop(['accum_size', 'remaining_vol'], axis=1, inplace=True) 
            bit_ask.reset_index(drop=True, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
            # Hedge trasnaction in orgin exchange (Kraken)

        else:
            
            bit_bid.rename(columns={'bid_size':'size','bid':'price'}, inplace=True)
            
            bit_bid = bit_bid.merge(pd.DataFrame(to_fill[['size','price']]).transpose(), 
                                    on=['size','price'], how='outer'
                                   ).sort_values(by='price', ascending=False, ignore_index=True)   
            
            bit_bid.rename(columns={'size':'bid_size','price':'bid'}, inplace=True)

            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
    else:
        
        # identify order's fee structure
        fee_type = 'taker' if to_fill['price'] < bit_bid.iloc[0]['bid'] else 'maker'            
        
        if fee_type == 'taker':
            
            bit_bid['accum_size'] = bit_bid['bid_size'].cumsum()


            vol_to_fill = to_fill['size']

            bit_bid['remaining_vol'] = vol_to_fill - bit_bid['accum_size']
            to_drop = bit_bid[bit_bid['remaining_vol']>0]

            bit_bid.drop(to_drop.index, inplace=True)
            bit_bid.reset_index(drop=True, inplace=True)

            new_vol = -bit_bid.iloc[0,-1] # remaing volume on surviving level

            bit_bid.iloc[0,0] = new_vol
            bit_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            bit_bid.reset_index(drop=True, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
            
            # Hedge trasnaction in orgin exchange (Kraken)
        else:
            bit_ask.rename(columns={'ask_size':'size','ask':'price'}, inplace=True)
            
            bit_ask = bit_ask.merge(pd.DataFrame(to_fill[['price','size']]).transpose(), 
                                    on=['price','size'], how='outer'
                                   ).sort_values(by='price', ascending=True, ignore_index=True) 
            
            bit_ask.rename(columns={'size':'ask_size','price':'ask'}, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)

C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [351]:
df_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074
3,0.011,28292.2,28305.5,0.123
4,1.221,28292.0,28307.3,0.204
...,...,...,...,...
95,3.726,28179.1,28430.4,0.001
96,3.059,28178.9,28431.0,0.001
97,3.733,28169.6,28431.8,3.840
98,2.756,28168.2,28432.1,3.032


In [350]:
bit_bid.iloc[0,:], bit_ask.iloc[0,:]

(bid_size        0.798
 bid         28285.300
 Name: 0, dtype: float64,
 ask         28289.000000
 ask_size        2.737301
 Name: 0, dtype: float64)

In [333]:
bit_ask

,ask,ask_size,price_x,size_x,price_y,size_y,price_x,size_x,price_y,size_y,...,price_x,size_x,price_y,size_y,price_x,size_x,price_y,size_y,price,size
0,28289,1.6393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28290,1.59568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28291,0.263632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28292,4.13541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28293,0.867221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,28294,2.93525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,28295,1.0663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,28296,5.42405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,28297,0.110386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,28298,2.19247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [295]:
levels_added

,size,price,type,transaction_time


In [288]:
bit_bid.shape

(25, 2)

In [297]:
bit_bid

,bid_size,bid,size_x,price_x,size_y,price_y,size_x,price_x,size_y,price_y,...,size_x,price_x,size_y,price_y,size_x,price_x,size_y,price_y,size,price
0,14.6057,28275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00512441,28273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0101581,28272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.003,28268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.58153,28266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.00089,28265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.30936,28264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.13373,28263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.596041,28262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.188323,28260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [296]:
bit_bid.shape

(38, 28)

In [289]:
bit_ask.shape

(25, 2)

In [289]:
bit_ask.shape

(25, 2)

### Asks

In [24]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [32]:
# merge levels to create new OB
asks = df_bit[['ask','ask_size']].merge(levels[['ask','ask_size']], how='outer', on='ask'
                                       ).sort_values(by='ask', ascending=True, ignore_index=True)

asks.rename(columns={'ask_size_x':'ask_size','ask_size_y':'add_size'}, inplace=True)

In [33]:
# identify each order's fee structure
asks['fee_type'] = ['taker' if ask < df_bit['bid'][0] else 'maker' if pd.notna(add_size) else np.nan
                    for ask, add_size in zip(asks['ask'].values, asks['add_size'].values)]

asks.fillna(0, inplace=True)

In [34]:
# compute total volume in each OB level
asks['new_size'] = asks['ask_size'].values+asks['add_size'].values

In [36]:
asks = asks[asks['new_size']!=0]

In [37]:
# Individual transaction's fee charge
#asks['fee'] = [.0015*ask*vol if fee_type=='maker' else .0030*ask*vol if fee_type=='taker' else 0 
#            for ask,vol,fee_type in zip(asks['ask'], asks['add_size'] ,asks['fee_type'])]

In [38]:
df_bit.shape[0],levels.shape[0], asks.shape[0]

(25, 30, 47)

___

In [60]:
new_df = df_bit.append(levels).reset_index(drop=True)

In [169]:
# create new orderbook
new_df = pd.concat([new_df[['bid_size', 'bid']].sort_values('bid', ascending=False).reset_index(drop=True), 
                   new_df[['ask', 'ask_size']].sort_values('ask', ascending=True).reset_index(drop=True)],
                   axis=1)

# Next steps

1. Insertar liquidez en OB:

    * Proceso de matching engine con ask subsecuente (órdenes límite que crucen el spread hasta que sean agotadas) (taker fee)
    * Calcular comisiones cada vez que se empaten órdenes
    * Calcular efectos de nuestras órdenes y crear un nuevo Orderbook
    
 
2. Comparar OB creado vs OB en siguiente timestamp  

    * Obtener diferencias en niveles para validar si se llenaron nuestras órdenes con el siguiente OB (maker fee)


3. Incorporar proceso de hedge en origin exchange

    * Cada vez que se ejecute una orden en destination, se hae una operación contraria a spot en el origin
    
    
4. Reflejar efectos de hedge en el origin exchange

    * Preguntar al profe